## Early Explorations

In [2]:
import os
import pandas as pd

meth_df = pd.read_excel('../../data/sample-TCGA-spreadsheets/TCGA-DATA-COMBINED.xlsx')
meth_df.head()

,ID_REF,LUSC_tumor,LUSC_normal,COAD_tumor,COAD_normal,GBM_tumor,GBM_normal
0,cg00000029,0.312090,0.277189,0.491594,0.201456,0.862540,0.625655
1,cg00000165,0.356855,0.152366,0.867351,0.236720,0.890708,0.223732
2,cg00000236,0.914564,0.874785,0.930446,0.877313,0.908234,0.924708
3,cg00000289,0.783759,0.788907,0.765557,0.765379,0.768677,0.716382
4,cg00000292,0.854914,0.754082,0.667273,0.599161,0.565240,0.567836


In [2]:
content = []
eul1db_srip_list = []
with open("../../data/eul1db-data/eul1db_srip.bed")as f:
    for line in f:
        content.append(line.strip().split())
for i, line in enumerate(content):
    if i < 3:
        continue
    eul1db_srip_list.append([line[0][3:], line[1], line[2], line[5]])
headers = ['Chr','Start','End','Strand']
eul1db_srip_df = pd.DataFrame(eul1db_srip_list, columns=headers)
eul1db_srip_df.head(n=5)

,Chr,Start,End,Strand
0,1,815511,815511,-
1,1,980069,980069,+
2,1,1149273,1149273,-
3,1,1687509,1687509,-
4,1,3053277,3053277,-


In [3]:
l1base_df = pd.read_excel('../../data/l1base/l1base.xlsx')
l1base_list = []

for index, row in l1base_df.iterrows():
    l1base_list.append([row[0], row[1], row[2], row[5]])

l1base_df = pd.DataFrame(l1base_list, columns=headers)
l1base_df.head(5)

,Chr,Start,End,Strand
0,8,134077776,134069730,-1
1,8,128460023,128451975,-1
2,8,72882593,72874543,-1
3,8,72154468,72146429,-1
4,8,91557655,91565704,1


In [3]:
# %load '../methylator/annotation/annotate_450k.py'

In [4]:
#annotate = load_obj('Annotate_450k') # load annotation object containing all 450k probes
annotate = Annotate_450k() # create annotation object containing probes 
save_obj(annotate, 'Annotate_450k') # save the 'annotate' dictionary to ./data/pickle
annotate.get_number()

482421

In [8]:
cg00050873 = annotate.probe['cg00050873']
cg00050873.cord

9363356

In [9]:
sorted_keys = annotate.get_sorted_probes_by_id()
sorted_keys[0], sorted_keys[210293]

('cg00000029', 'cg11149743')

In [10]:
cg00000029 = annotate.get_probe(sorted_keys[0])
cg27666046 = annotate.get_probe(sorted_keys[210293])

In [11]:
cg00000029.cord, cg27666046.cord, cg00000029.tour

(53468112, 46688419, 'N_Shore')

In [12]:
probe_list = annotate.get_probes_by_list(sorted_keys[0:5]) # get first 5 probes
for probe in probe_list: # sorting on probe id does not follow by chr in an expected way. 
    print ('ID: {p}, Chr: {c}'.format(p=probe.id, c=probe.chr)) 

ID: cg00000029, Chr: 16
ID: cg00000108, Chr: 3
ID: cg00000109, Chr: 3
ID: cg00000165, Chr: 1
ID: cg00000236, Chr: 8


In [13]:
sorted_by_chr = annotate.get_sorted_probes_by_chr()
chromosome1 = sorted_by_chr[0:46566] # huh?
chromosome1[0].cord, chromosome1[46565].cord

(91194674, 220863462)

In [14]:
# Return probes in the vicinity of a gene
probe_refs = annotate.get_probe_refs_by_gene('SRPK1')
probe_refs[0]

'cg00087417'

In [15]:
# Return probes in vicinity of a gene (i.e. SRPK1)
probes_from_gene = annotate.get_probes_by_gene('SRPK1')
print ('SRPK1 probe found on Chr {p}.'.format(p=probes_from_gene[0].chr))

SRPK1 probe found on Chr 6.


In [16]:
# Return probes in vicinity of a feature (i.e. BODY, TSS200, TSS1500, UTR5, UTR3)
BODY_probes = annotate.get_probes_by_location(Location.BODY) # 
TSS200_probes = annotate.get_probes_by_location(Location.TSS200) #
TSS1500_probes = annotate.get_probes_by_location(Location.TSS1500) #
UTR3_probes = annotate.get_probes_by_location(Location.UTR3) #
UTR5_probes = annotate.get_probes_by_location(Location.UTR5) #

In [17]:
print (len(BODY_probes), len(TSS1500_probes), len(UTR5_probes), len(TSS200_probes), len(UTR3_probes))

174438 84335 65476 62625 19694


In [18]:
meth_df.head()
for index, row in meth_df.head().iterrows():
    print (index, row[0])

0 cg00000029
1 cg00000165
2 cg00000236
3 cg00000289
4 cg00000292


In [6]:
methyl_probe_dict = {}

for index, row in meth_df.head(5).iterrows():
    methyl_probe_dict[row[0]] = annotate.probe.get(row[0]) # row[0] == 'cg00000029' i.e. key value
    setattr(methyl_probe_dict[row[0]], 'LUSC_tumor',  row[1]) # add spreadsheet attributes 
    setattr(methyl_probe_dict[row[0]], 'LUSC_normal', row[2]) # from TCGA-DATA-COMBINED.xlsx...
    setattr(methyl_probe_dict[row[0]], 'COAD_tumor',  row[3])
    setattr(methyl_probe_dict[row[0]], 'COAD_normal', row[4])
    setattr(methyl_probe_dict[row[0]], 'GBM_tumor',   row[5])
    setattr(methyl_probe_dict[row[0]], 'GBM_normal',  row[6])

In [7]:
cg00000292 = methyl_probe_dict['cg00000292'] # get a 'methyl' probe by name

In [22]:
cg00000292.cord, cg00000292.refseq, cg00000292.COAD_tumor, cg00000292.COAD_normal

(28890100, 'NM_004320;NM_173201', 0.667273112085446, 0.599161178855325)

In colorectal cancers 600-800 genes are transcriptionally-silenced by CpG island methylation compared to normal tissue. Transcriptional silencing may be more important than mutation in causing gene silencing in progression to cancer.

About 70% of human promoters located near the transcription start site of a gene contain a CpG island.

In [27]:
# Remove SNPs.. (do this after the cells above)
#annotate.remove_snp_probes()
all_probes = annotate.get_all_probes()
annotate.get_number()

482421